In [1]:
import torch
torch.cuda.empty_cache() 

In [2]:
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2
import urllib.request
from PIL import Image

In [3]:
import pandas as pd
import pyproj

In [4]:
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [5]:
from pathlib import Path

In [6]:
import joblib

In [7]:
from mt_function_yl import *

In [8]:
import seaborn as sn

In [9]:
from scipy import stats

In [10]:
from tabulate import tabulate

In [11]:
import statsmodels.api as sm

### Data loading

In [ ]:
# For 2022

In [16]:
ca2022 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/sid_dav_verkehrszaehlung_miv_od2031_2022.csv')
bi2022 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/2022_verkehrszaehlungen_werte_fussgaenger_velo.csv')

/sctmp/yelhe/ipykernel_2187633/489067175.py:1: DtypeWarning: Columns (5,14) have mixed types. Specify dtype option on import or set low_memory=False.
  ca2022 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/sid_dav_verkehrszaehlung_miv_od2031_2022.csv')


In [49]:
# ca2022.info()

In [51]:
ca2022['D1ID'].unique()

array([ 2,  1, 13, 14, 16, 18, 22, 21,  3, 23, 20, 11, 19, 15, 12,  4,  8,
        6, 17, 29, 27, 10, 26,  5,  7,  9])

In [44]:
# ca2022.groupby(['MSID', 'MessungDatZeit']).size().reset_index().rename(columns={0:'count'})

In [45]:
caw = ca2022.copy(deep = True)
caw.loc[:, 'Date'] = pd.to_datetime(caw.loc[:, 'MessungDatZeit'], 
                                              format = "%Y-%m-%dT%H:%M:%S",
                                              errors='coerce')
caw.loc[:,'year'] = caw.loc[:,'Date'].dt.year
caw.loc[:,'month'] = caw.loc[:,'Date'].dt.month
caw.loc[:,'day'] = caw.loc[:,'Date'].dt.day
caw.loc[:, 'week'] = caw.loc[:, 'Date'].dt.isocalendar().week

In [52]:
caws = caw.groupby(['MSID', 'year', 'month', 'day', 'week' ])['AnzFahrzeuge'].sum().reset_index()

In [113]:
caws

,MSID,year,month,day,week,AnzFahrzeuge
0,Z001M001,2022,1,1,52,3903.0
1,Z001M001,2022,1,2,52,4034.0
2,Z001M001,2022,1,3,1,5350.0
3,Z001M001,2022,1,4,1,5795.0
4,Z001M001,2022,1,5,1,5844.0
...,...,...,...,...,...,...
66425,Z097M002,2022,12,27,52,6647.0
66426,Z097M002,2022,12,28,52,6862.0
66427,Z097M002,2022,12,29,52,6814.0
66428,Z097M002,2022,12,30,52,7318.0


In [114]:
cawsm = caws.groupby(['MSID', 'year', 'month', 'week' ])['AnzFahrzeuge'].sum().reset_index()

In [115]:
cawsm

,MSID,year,month,week,AnzFahrzeuge
0,Z001M001,2022,1,1,38392.0
1,Z001M001,2022,1,2,41295.0
2,Z001M001,2022,1,3,42899.0
3,Z001M001,2022,1,4,43702.0
4,Z001M001,2022,1,5,6148.0
...,...,...,...,...,...
11461,Z097M002,2022,12,48,37068.0
11462,Z097M002,2022,12,49,65870.0
11463,Z097M002,2022,12,50,64623.0
11464,Z097M002,2022,12,51,60259.0


In [116]:
# cawsm.to_csv('/home/yelhe/script/mt/output/car_tc_2022.csv', index = False)

In [119]:
cawsmm = caws.groupby(['year', 'month', 'week' ])['AnzFahrzeuge'].mean().reset_index()
# cawsmm.to_csv('/home/yelhe/script/mt/output/car_tc_2022_weekmean.csv', index = False)

In [60]:
bi2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1189238 entries, 0 to 1189237
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   FK_STANDORT  1189238 non-null  int64  
 1   DATUM        1189238 non-null  object 
 2   VELO_IN      693598 non-null   float64
 3   VELO_OUT     594522 non-null   float64
 4   FUSS_IN      495640 non-null   float64
 5   FUSS_OUT     495640 non-null   float64
 6   OST          1189238 non-null  int64  
 7   NORD         1189238 non-null  int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 72.6+ MB


In [66]:
bi2022.head()

,FK_STANDORT,DATUM,VELO_IN,VELO_OUT,FUSS_IN,FUSS_OUT,OST,NORD
0,2979,2022-01-01T00:00,0.0,0.0,NaN,NaN,2681858,1251991
1,2978,2022-01-01T00:00,0.0,0.0,NaN,NaN,2683406,1251617
2,2641,2022-01-01T00:00,NaN,NaN,3.0,0.0,2682813,1245490
3,3598,2022-01-01T00:00,0.0,0.0,NaN,NaN,2684006,1246566
4,3020,2022-01-01T00:00,NaN,NaN,3.0,3.0,2680439,1249930


In [64]:
bi22 = bi2022.drop(['FUSS_IN', 'FUSS_OUT', 'OST', 'NORD'], axis = 1)

In [65]:
bi22 = bi22.fillna(0)

In [73]:
bi22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1189238 entries, 0 to 1189237
Data columns (total 5 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   FK_STANDORT  1189238 non-null  int64  
 1   DATUM        1189238 non-null  object 
 2   VELO_IN      1189238 non-null  float64
 3   VELO_OUT     1189238 non-null  float64
 4   velo         1189238 non-null  float64
dtypes: float64(3), int64(1), object(1)
memory usage: 45.4+ MB


In [76]:
bi22['velo'] = (bi22['VELO_IN'] + bi22['VELO_OUT']) * 1.185

In [78]:
bi22['velo'].sum()

12142421.264999995

In [79]:
biw = bi22.copy(deep = True)
biw.loc[:, 'Date'] = pd.to_datetime(biw.loc[:, 'DATUM'], 
                                              format = "%Y-%m-%dT%H:%M",
                                              errors='coerce')
biw.loc[:,'year'] = biw.loc[:,'Date'].dt.year
biw.loc[:,'month'] = biw.loc[:,'Date'].dt.month
biw.loc[:,'day'] = biw.loc[:,'Date'].dt.day
biw.loc[:, 'week'] = biw.loc[:, 'Date'].dt.isocalendar().week

In [121]:
biwsm = biw.groupby(['year', 'month', 'week' ])['velo'].sum().reset_index()
# biwsm.to_csv('/home/yelhe/script/mt/output/bic_tc_2022_weekmean.csv', index = False)

In [80]:
biws = biw.groupby(['FK_STANDORT', 'year', 'month', 'week' ])['velo'].sum().reset_index()

In [103]:
# bicycle counting station locations
bcl = pd.read_csv('/home/yelhe/data/Traffic_network_variables/countstation_bicycle.csv')

In [105]:
bcln = bcl[['fk_zaehler','id1']]

In [106]:
bcln = bcln.rename(columns = {'id1': 'FK_STANDORT'})

In [107]:
biws_id = pd.merge(biws, bcln, how = 'left', on = ['FK_STANDORT'])

In [108]:
biws_id

,FK_STANDORT,year,month,week,velo,fk_zaehler
0,20,2022,1,1,0.0,U15G3063864
1,20,2022,1,2,0.0,U15G3063864
2,20,2022,1,3,0.0,U15G3063864
3,20,2022,1,4,0.0,U15G3063864
4,20,2022,1,5,0.0,U15G3063864
...,...,...,...,...,...,...
2165,5004,2022,12,48,0.0,U15G3063873
2166,5004,2022,12,49,0.0,U15G3063873
2167,5004,2022,12,50,0.0,U15G3063873
2168,5004,2022,12,51,0.0,U15G3063873


In [110]:
# biws_id.to_csv('/home/yelhe/script/mt/output/bic_tc_2022_new.csv', index = False)

In [112]:
biws_id['fk_zaehler'].unique()

array(['U15G3063864', 'Y2G13124879', 'U15G3104447', 'Y2H16069942',
       'Y2H16069943', 'U15G3063865', 'U15G6121229', 'U15G3104443',
       'U15G3063870', 'ECO09113507', 'ECO10053914', 'ECO09113500',
       'Y2H19070283', 'Y2H19070585', 'Y2H19090841', 'Y2H19101255',
       'Y2H19111476', 'Y2H19111477', 'YPH15108550', 'Y2H20011946',
       'Y2H19101198', 'U15G3063872', 'Y2H20063173', 'Y2H20083483',
       'Y2H20114444', 'YSH20124740', 'U15G3063873', 'U15G3063869',
       'YSH18065716', 'Y2H18106792', 'ECO09113503', 'ECO09113499',
       'Y2H21056105', 'Y2H21056106', 'U15G3063866', 'DGP22022111',
       'U15G3104444', 'Y2H21111102', 'U15G3063867', 'Y2H22063445',
       'Y2H21015036', 'Y2H22073807'], dtype=object)

In [89]:
biwc_v = biws.loc[biws['velo'] != 0]

In [90]:
biwc_v

,FK_STANDORT,year,month,week,velo
63,60,2022,1,1,4111.950
64,60,2022,1,2,4143.945
65,60,2022,1,3,4670.085
66,60,2022,1,4,4952.115
67,60,2022,1,5,549.840
...,...,...,...,...,...
2141,5002,2022,12,48,2638.995
2142,5002,2022,12,49,5244.810
2143,5002,2022,12,50,2719.575
2144,5002,2022,12,51,3584.625


In [99]:
biwc_mean = biwc_v.groupby(['week' ])['velo'].mean().reset_index()

In [100]:
biwc_mean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   week    52 non-null     UInt32 
 1   velo    52 non-null     float64
dtypes: UInt32(1), float64(1)
memory usage: 808.0 bytes


In [101]:
# biwc_mean.to_csv('/home/yelhe/script/mt/output/bic_tc_2022_mean.csv', index = False)

In [375]:
aa = pd.read_csv('/home/yelhe/script/mt/dataraw/dfaa.csv')

In [376]:
# tt variables

# Distance to features
busl = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_busline.csv')
traml = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_tramrail.csv')
trainl = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_sbahn_h.csv')
plight = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_publiclight.csv')
station = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_station_h.csv')
parkcar = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_parking_car_space.csv')
parktw = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_parking_twowheeler.csv')

pedstrainc = pd.read_csv('/home/yelhe/data/Traffic_network_variables/acc_pedestraincrossing_n.csv') # Greg
stop = pd.read_csv('/home/yelhe/data/Traffic_network_variables/acc_stopsign_n.csv') # Greg

# Road width & surface information
roadinfo = pd.read_csv('/home/yelhe/data/Traffic_network_variables/acc_roadsurfacewidth.csv') # Greg
# Speedlimit
speedlimit = pd.read_csv('/home/yelhe/data/Traffic_network_variables/acc_speedlimit.csv') # Greg

# Traffic volume/exposure
gvm = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_gvm.csv')

# Area statistics zone
trafficarea = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_trafficarea_h.csv')
statzone = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_statzonen_h.csv')

# Curb info
curb = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_curb.csv')



In [377]:
# Traffic count
bc = pd.read_excel('/home/yelhe/data/Traffic_network_variables/traffic_count/bicyclecount_1.xlsx')
vc = pd.read_excel('/home/yelhe/data/Traffic_network_variables/traffic_count/carcount_1.xlsx')
# bc = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bc_nu.csv')
# vc = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/vc_nu.csv')

In [378]:
ttv = pd.DataFrame()

In [379]:
buslg = busl.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})

In [380]:
buslg = buslg.rename(columns = {'distance': 'dbusl'})
buslg = buslg.drop(['count'], axis = 1)

In [381]:
ttv['Nr'] = aa['Nr']

In [382]:
ttv = ttv.merge(buslg, on = ['Nr'])

In [383]:
tramlg = traml.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
tramlg = tramlg.rename(columns = {'distance': 'dtraml'})
tramlg = tramlg.drop(['count'], axis = 1)

In [384]:
ttv = ttv.merge(tramlg, on = ['Nr'])

In [385]:
trainlg = trainl.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
trainlg = trainlg.rename(columns = {'distance': 'dtrainl'})
trainlg = trainlg.drop(['count'], axis = 1)

In [386]:
ttv = ttv.merge(trainlg, on = ['Nr'])

In [387]:
plightg = plight.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
plightg = plightg.rename(columns = {'distance': 'dplight'})
plightg = plightg.drop(['count'], axis = 1)

In [388]:
ttv = ttv.merge(plightg, on = ['Nr'])

In [389]:
stationg = station.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
stationg = stationg.rename(columns = {'distance': 'dstation'})
stationg = stationg.drop(['count'], axis = 1)

In [390]:
ttv = ttv.merge(stationg, on = ['Nr'])

In [391]:
parkcarg = parkcar.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
parkcarg = parkcarg.rename(columns = {'distance': 'dparkcar'})
parkcarg = parkcarg.drop(['count'], axis = 1)
ttv = ttv.merge(parkcarg, on = ['Nr'])
parktwg = parktw.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
parktwg = parktwg.rename(columns = {'distance': 'dparktw'})
parktwg = parktwg.drop(['count'], axis = 1)
ttv = ttv.merge(parktwg, on = ['Nr'])
pedcrg = pedstrainc.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
pedcrg = pedcrg.rename(columns = {'distance': 'dpedcro'})
pedcrg = pedcrg.drop(['count'], axis = 1)
ttv = ttv.merge(pedcrg, on = ['Nr'])
stopsg = stop.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
stopsg = stopsg.rename(columns = {'distance': 'dstopsign'})
stopsg = stopsg.drop(['count'], axis = 1)
ttv = ttv.merge(stopsg, on = ['Nr'])

In [392]:
curbg = curb.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
curbg = curbg.rename(columns = {'distance': 'dcurb'})
curbg = curbg.drop(['count'], axis = 1)
ttv = ttv.merge(curbg, on = ['Nr'])

In [393]:
curbty = curb[['Nr', 'distance', 'abschluss']]

In [394]:
curbty = curbty.rename(columns = {'abschluss':'curbtype', 'distance':'dcurb'})
curbty = curbty.drop_duplicates()
ttv = ttv.merge(curbty, on = ['Nr', 'dcurb'])

In [395]:
gvmg = gvm.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
gvmg = gvmg.rename(columns = {'distance': 'dgvm'})
gvmg = gvmg.drop(['count'], axis = 1)
ttv = ttv.merge(gvmg, on = ['Nr'])
gvmvalue = gvm[['Nr', 'DWV_ANZ_Q', 'MSP_ANZ_Q', 'ASP_ANZ_Q', 'distance']]
gvmvalue = gvmvalue.rename(columns = {'DWV_ANZ_Q':'gvm_dwv',
                                      'MSP_ANZ_Q':'gvm_msp',
                                      'ASP_ANZ_Q':'gvm_asp',
                                      'distance':'dgvm'})
ttv = ttv.merge(gvmvalue, on = ['Nr', 'dgvm'])

In [396]:
ttv.columns

Index(['Nr', 'dbusl', 'dtraml', 'dtrainl', 'dplight', 'dstation', 'dparkcar',
       'dparktw', 'dpedcro', 'dstopsign', 'dcurb', 'curbtype', 'dgvm',
       'gvm_dwv', 'gvm_msp', 'gvm_asp'],
      dtype='object')

In [397]:
# ttv = ttv.drop(['curbtype'], axis = 1)
ttv = ttv.drop_duplicates()

In [398]:
# ttv = ttv.rename(columns = {'curbtype_y': 'curbtype'})

In [399]:
ttv['curbtype'].unique()

array([25, 15])

In [400]:
ttv = ttv.reset_index()

In [401]:
ttv = ttv.drop(['index'], axis = 1)

In [402]:
print(len(statzone['stznr'].unique()),
      len(statzone['qnr'].unique()),
      len(statzone['knr'].unique()))

122 33 12


In [403]:
statzoneg = statzone[['Nr', 'stznr', 'qnr', 'knr']]
statzoneg = statzoneg.rename(columns = {'stznr':'z_stznr', 'qnr':'z_qnr', 'knr':'z_knr'})
ttv = ttv.merge(statzoneg, on = ['Nr'])

In [404]:
trafficarea.columns

Index(['Nr', 'Koord E', 'Koord N', 'lat', 'lon', 'id', 'zonenname', 'zonentyp',
       'zonentyp_t', 'umgesetzt_', 'rechtskrae', 'publiziert', 'objectid',
       'distance'],
      dtype='object')

In [405]:
trafficarea.groupby(['zonentyp', 'zonentyp_t']).size()

zonentyp        zonentyp_t
Begegnungszone  T20            34
Fussgängerzone  T0             32
T30-Zone        T30           284
dtype: int64

In [406]:
tareag = trafficarea[['Nr', 'zonentyp_t', 'distance']]
tareag = tareag.rename(columns = {'zonentyp_t':'trafficarea',
                                  'distance':'dtrafficarea'})
ttv = ttv.merge(tareag, on = ['Nr'])

In [407]:
rig = roadinfo[['Nr', 'road_width', 'surface', 'distance']]
rig = rig.rename(columns = {'road_width':'r_width',
                               'distance':'droad',
                               'surface':'r_surface'})
ttv = ttv.merge(rig, on = ['Nr'])

In [408]:
slg = speedlimit[['Nr', 'speed', 'distance']]
slg = slg.rename(columns = {'speed':'speedlimit',
                            'distance':'droad_sl'})
ttv = ttv.merge(slg, on = ['Nr'])

In [409]:
ttv.columns

Index(['Nr', 'dbusl', 'dtraml', 'dtrainl', 'dplight', 'dstation', 'dparkcar',
       'dparktw', 'dpedcro', 'dstopsign', 'dcurb', 'curbtype', 'dgvm',
       'gvm_dwv', 'gvm_msp', 'gvm_asp', 'z_stznr', 'z_qnr', 'z_knr',
       'trafficarea', 'dtrafficarea', 'r_width', 'r_surface', 'droad',
       'speedlimit', 'droad_sl'],
      dtype='object')

In [410]:
# ttv.to_csv('/home/yelhe/data/Traffic_network_variables/ttvariables_sum_new.csv', index = False)

In [411]:
# ttv = pd.read_csv('/home/yelhe/data/Traffic_network_variables/ttvariables_sum_new.csv')

In [412]:
bc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Unnamed: 0             350 non-null    int64         
 1   Nr                     350 non-null    int64         
 2   FK_ZAEHLER             350 non-null    object        
 3   distance               350 non-null    float64       
 4   Datum                  185 non-null    datetime64[ns]
 5   Zeit                   185 non-null    object        
 6   Date                   185 non-null    datetime64[ns]
 7   year                   185 non-null    float64       
 8   month                  185 non-null    float64       
 9   day                    185 non-null    float64       
 10  week                   185 non-null    float64       
 11  bicycle_traffic_count  350 non-null    float64       
 12  date                   117 non-null    datetime64[ns]
 13  OST  

In [413]:
bc = bc.drop(['Unnamed: 0'], axis = 1)

In [262]:
# list = bc1921.index.values

In [263]:
# bc1921 = bc1921.drop(index = [76, 97, 79, 73, 139, 86, 149, 128, 118])

In [414]:
bcg = bc.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
bcg = bcg.rename(columns = {'distance': 'd_bicyclecount'})
bcg = bcg.drop(['count'], axis = 1)

In [415]:
bcg

,Nr,d_bicyclecount
0,99900044963840,274.458564
1,99900045510756,566.991776
2,99900045511204,25.440177
3,99900045553013,187.579285
4,99900045612893,484.723513
...,...,...
345,99900063695545,62.728320
346,99900063698497,310.789294
347,99900063992212,236.886674
348,99900064009149,340.463140


In [416]:
# bcg = bcg.drop_duplicates()
ttv = ttv.merge(bcg, on = ['Nr'], how = 'left')
bcgv = bc[['Nr', 'bicycle_traffic_count', 'distance']]
bcgv = bcgv.rename(columns = {'bicycle_traffic_count':'bicyclecount',
                              'distance':'d_bicyclecount'})


In [417]:
bcgv

,Nr,bicyclecount,d_bicyclecount
0,99900060852597,11188.80428,641.292963
1,99900063018611,11188.80428,959.103749
2,99900052723510,11188.80428,646.572037
3,99900061514043,11188.80428,397.277552
4,99900063037843,11188.80428,604.035921
...,...,...,...
345,99900055905276,6692.00841,2405.574172
346,99900060670975,11188.80428,2543.971891
347,99900047367168,11188.80428,2661.635653
348,99900050714619,11188.80428,2831.090872


In [418]:
ttv.columns

Index(['Nr', 'dbusl', 'dtraml', 'dtrainl', 'dplight', 'dstation', 'dparkcar',
       'dparktw', 'dpedcro', 'dstopsign', 'dcurb', 'curbtype', 'dgvm',
       'gvm_dwv', 'gvm_msp', 'gvm_asp', 'z_stznr', 'z_qnr', 'z_knr',
       'trafficarea', 'dtrafficarea', 'r_width', 'r_surface', 'droad',
       'speedlimit', 'droad_sl', 'd_bicyclecount'],
      dtype='object')

In [419]:
ttv = ttv.merge(bcgv, on = ['Nr', 'd_bicyclecount'], how = 'left')

In [420]:
ttv['d_bicyclecount']

0       641.292963
1       959.103749
2       646.572037
3       397.277552
4       604.035921
          ...     
345    2405.574172
346    2543.971891
347    2661.635653
348    2831.090872
349    2952.945811
Name: d_bicyclecount, Length: 350, dtype: float64

In [197]:
# vc1921 = vc1921.drop(index = [42, 40, 76])

In [198]:
# vc1921.loc[41, 'car_traffic_count'] = 88424.5

In [199]:
# vc1921.loc[39, 'car_traffic_count'] = 87416.5

In [200]:
# vc1921.loc[75, 'car_traffic_count'] = 88582.5

In [421]:
vcg = vc.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
vcg = vcg.rename(columns = {'distance': 'd_carcount'})
vcg = vcg.drop(['count'], axis = 1)
vcg = vcg.drop_duplicates()
ttv = ttv.merge(vcg, on = ['Nr'], how = 'left')
vcgv = vc[['Nr', 'car_traffic_count', 'distance']]
vcgv = vcgv.rename(columns = {'car_traffic_count':'carcount',
                              'distance':'d_carcount'})
ttv = ttv.merge(vcgv, on = ['Nr', 'd_carcount'], how = 'left')

In [423]:
ttv.to_csv('/home/yelhe/data/Traffic_network_variables/ttvariables_sum_withtrafficcount_final.csv', index = False)

In [427]:
# ttv = pd.read_csv('/home/yelhe/data/Traffic_network_variables/ttvariables_sum_withtrafficcount.csv')
vfpath = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_velofusspath.csv')

In [428]:
vfpg = vfpath.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
vfpg = vfpg.rename(columns = {'distance': 'dvfpath'})
vfpg = vfpg.drop(['count'], axis = 1)

In [429]:
vfpg

,Nr,dvfpath
0,99900044963840,9.410807
1,99900045510756,0.420799
2,99900045511204,0.437524
3,99900045553013,6.802020
4,99900045612893,7.188158
...,...,...
345,99900063695545,5.926033
346,99900063698497,0.909111
347,99900063992212,4.419567
348,99900064009149,2.945314


In [430]:
ttv = ttv.merge(vfpg, on = ['Nr'])

In [431]:
ttv.to_csv('/home/yelhe/data/Traffic_network_variables/ttvariables_sum_final.csv', index = False)

In [432]:
ttv.columns

Index(['Nr', 'dbusl', 'dtraml', 'dtrainl', 'dplight', 'dstation', 'dparkcar',
       'dparktw', 'dpedcro', 'dstopsign', 'dcurb', 'curbtype', 'dgvm',
       'gvm_dwv', 'gvm_msp', 'gvm_asp', 'z_stznr', 'z_qnr', 'z_knr',
       'trafficarea', 'dtrafficarea', 'r_width', 'r_surface', 'droad',
       'speedlimit', 'droad_sl', 'd_bicyclecount', 'bicyclecount',
       'd_carcount', 'carcount', 'dvfpath'],
      dtype='object')

In [435]:
# ttv['speedlimit'].unique()
len(ttv['z_qnr'].unique())

33

In [434]:
ttv['speedlimit'].unique()

array([30.,  0., 60., 50., 80., 20.])